# Lab 8 - Combining Attendance and Practice Quiz Attempts

## Background

In all of my class, I use an attendace quiz to track student attendance.  Note that students take multiple attempts at the same quiz, one per class; so that number of attempts a student takes on this quiz represents the number of class session that student has attended.

In some, but not all, of my courses I also provide practice quizzes that students can use to prepare for actual quizzes and tests.  These quizzes pull questions randomly from a bank of questions, allow students unlimited attempts, and are not used as part of the students grade.

In this lab, you will collect simulated data from mock classes into one table and in the next lab you will summarize these data.

## Tasks 

The files found in `attendance_example.zip` contains (made-up and random) examples of the D2L files that I use to summarize my attendance quizzes and practice quizzes
Make sure you download `attendance_example.zip` to the `data` folder inside the course repository, then unzip the file.

1. Use `glob` to find the path to all csv files.
2. Use write functions that use regular expressions to extract the class name, quiz type (`Attendance` and `Practice`), and the module number (if the file is a practice quiz.
3. Write a function that takes a path as an argument and returns a dataframe that contains:
    * All of the original columns
    * A Class column that holds the class identifier
    * A Category column that contains the quiz type
    * A Module column that (a) contains the module number for a practice quiz, or (b) is otherwise empty.
4. Use a loop, `union`, and the accumulator pattern to load all of the data into a single table.
5. Write the resulting table to a csv file.

In [46]:
from glob import glob
files = glob('./data/attendance_example/*/*.csv')
files

['./data/attendance_example/dsci494s7/Attendance Quiz - User Attempts.csv',
 './data/attendance_example/dsci494s7/Practice Quiz - Module 1 - User Attempts.csv',
 './data/attendance_example/dsci494s7/Practice Quiz - Module 2 - User Attempts.csv',
 './data/attendance_example/dsci494s7/Practice Quiz - Module 3 - User Attempts.csv',
 './data/attendance_example/dsci494s7/Practice Quiz - Module 4 - User Attempts.csv',
 './data/attendance_example/stat180s18/Attendance Quiz - User Attempts.csv',
 './data/attendance_example/stat491s1/Attendance Quiz - User Attempts.csv',
 './data/attendance_example/stat491s1/Practice Quiz - Module 1 - User Attempts.csv',
 './data/attendance_example/stat491s1/Practice Quiz - Module 2 - User Attempts.csv',
 './data/attendance_example/stat491s1/Practice Quiz - Module 3 - User Attempts.csv',
 './data/attendance_example/stat491s1/Practice Quiz - Module 4 - User Attempts.csv']

In [47]:
import re
CLASS_NAME_RE = re.compile(r'^\./data/attendance_example/([a-z]*[0-9]*).*\.csv$')
class_name = lambda p: CLASS_NAME_RE.match(p).group(1) 
class_names = lambda files: [class_name(p) for p in files]
class_names(files)

['dsci494',
 'dsci494',
 'dsci494',
 'dsci494',
 'dsci494',
 'stat180',
 'stat491',
 'stat491',
 'stat491',
 'stat491',
 'stat491']

In [99]:
QUIZ_NAME_RE = re.compile(r'^\./data/attendance_example/.*/([A-Z][a-z]*).*\.csv$')
quiz_name = lambda p: QUIZ_NAME_RE.match(p).group(1) 
quiz_names = lambda files: [quiz_name(p) for p in files]
quiz_names(files)

['Attendance',
 'Practice',
 'Practice',
 'Practice',
 'Practice',
 'Attendance',
 'Attendance',
 'Practice',
 'Practice',
 'Practice',
 'Practice']

In [208]:
MODULE_NAME_RE = re.compile(r'^\./data/attendance_example/.*/Practice.*([0-5]).*\.csv$')
module_name = lambda p: MODULE_NAME_RE.match(p).group(1)
#module_check = lambda path: ifelse(module_name(path) is None, "", module_name(path))
match = lambda p:MODULE_NAME_RE.match(p)
module_names = lambda files: [module_name(p) for p in files if quiz_name(p) == 'Practice']
module_names(files)
#match('./data/attendance_example/dsci494s7/Attendance Quiz - User Attempts.csv')
#module_name('./data/attendance_example/dsci494s7/Practice Quiz - Module 3 - User Attempts.csv')

['1', '2', '3', '4', '1', '2', '3', '4']

In [85]:
import pandas as pd
from dfply import *
df = {path:pd.read_csv(path) for path in files}
df['./data/attendance_example/dsci494s7/Attendance Quiz - User Attempts.csv'].head()

,Org Defined ID,UserName,FirstName,LastName,Attempt #,Score,Out Of,Attempt_Start,Attempt_End,Percent
0,14460432,au9747cp,Jericho,Greer,1,1,1,2019-01-14 14:00:00,2019-01-14 14:06:00,100 %
1,14460432,au9747cp,Jericho,Greer,2,1,1,2019-01-16 14:00:00,2019-01-16 14:08:00,100 %
2,14460432,au9747cp,Jericho,Greer,3,1,1,2019-01-18 14:00:00,2019-01-18 14:05:00,100 %
3,14460432,au9747cp,Jericho,Greer,4,1,1,2019-01-23 14:00:00,2019-01-23 14:06:00,100 %
4,14460432,au9747cp,Jericho,Greer,5,1,1,2019-01-25 14:00:00,2019-01-25 14:10:00,100 %


In [198]:
from more_dfply import ifelse
def createdf(path):
    return df[path] >> mutate(Class = class_name(path), Category = quiz_name(path), 
                              Module = ifelse(quiz_name(path)=='Practice', module_name(path), ''))

In [211]:
createdf('./data/attendance_example/dsci494s7/Attendance Quiz - User Attempts.csv').head()

AttributeError: 'NoneType' object has no attribute 'group'

In [215]:
createdf('./data/attendance_example/dsci494s7/Practice Quiz - Module 1 - User Attempts.csv')

TypeError: object of type 'bool' has no len()

In [213]:
#alldata = {f:createdf(f) for f in files}

In [214]:
#for d in alldata.values():
#    df = df >> union_all(d)

In [ ]:
df

In [ ]:
df.to_csv("attendance.csv", index = False)